In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

%matplotlib inline

import io, os, glob, gc
import timeit
from nbformat import current

HISTOGRAM_BITS_PER_COLOR = 2

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

execute_notebook('cuav.ipynb')

ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_COUNT_THRESHOLD = 30

SHOW_DUPES = True
#MIN_DEDUPE_REGIONS = 0
#DEDUPE_REGIONS = False
#REGION_MERGE_PIXEL_DISTANCE = 50
#MIN_REGION_SCORE = 0

/home/johnchu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/johnchu/anaconda2/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [2]:
def ProcessFile(image):
        bin = InitBin(image)
        lut = InitLUT(bin)
        qimage = quantize(image, lut)

        hist = BuildHistogram(qimage)
        mask = HistogramThreshold(qimage, hist)

        regions = AssignRegions(mask)
        ScoreRegions(regions, qimage, hist)
        print 'Total regions: %i' % regions.num_regions
        PruneRegions(regions)
        
        del bin,lut,qimage,hist,mask
        gc.collect()
        
        return regions

In [3]:
#from guppy import hpy
#hp = hpy()
#before = hp.heap()

## critical section here
#raw = cv2.imread('batch/few/1.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
#regions = ProcessFile(image)

#after = hp.heap()
#leftover = after - before
##import pdb; pdb.set_trace()

#print leftover.byclass

In [4]:
#raw = cv2.imread('batch/raw/image_car_full.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)

#%prun regions = ProcessFile(image)

In [5]:
#for dirname, dirnames, filenames in os.walk('batch/few/'):
#    for filename in sorted(filenames):
#        print(os.path.join(dirname, filename))


In [6]:
PrintSettings()

raw_files_dir = 'batch/test/'
proc_files_dir = raw_files_dir + 'results/'

if not os.path.exists(proc_files_dir):
    os.makedirs(proc_files_dir)
    
for f in sorted(glob.glob(raw_files_dir + '*.*')):
    file = f.replace(raw_files_dir,'')
    if file.lower().endswith('.jpg') or file.lower().endswith(('png')):
        print file
        raw = cv2.imread(f)
        image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
        
        regions = ProcessFile(image)
        
        pruned_image = np.copy(image)
        MarkRegions(pruned_image, regions)
        bgr = cv2.cvtColor(pruned_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(proc_files_dir + file, bgr)
        print '\n'
    else:
        print 'Skipping %s' % file

Adaptive Threshold Disabled with count set at 30 

HISTOGRAM_BITS_PER_COLOR = 2
NUM_BINS = 4
COLOR_SPECTRUM = 65
ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_CEILING_THRESHOLD = 1500
HISTOGRAM_FLOOR_THRESHOLD = 50
SHOW_DUPES = True
DEDUPE_REGIONS = True
MIN_DEDUPE_REGIONS = 5
REGION_MERGE_PIXEL_DISTANCE = 20
MIN_REGION_SCORE = 0

raw2015103000291033Z.png
min_r=18, max_r=255, bin_spacing.r=60
min_g=18, max_g=255, bin_spacing.g=60
min_b=24, max_b=255, bin_spacing.b=58
image height=2048,width=2448,depth=3
tt size: 5013298
tt size: 5013298
Total regions: 127
Min Region Score: 29
Average Region Score: 74
Median Region Score: 0
Max Region Score: 941
Total Pruned Regions: 22
    Region Score  Region Pixel Avg  Mod  Div Region Active
0            225                22    5   10         False
1            867                32    3   27         False
2             29                30   29    0         False
3            794                 8    2   99          True
4             29          

In [7]:
%sx ipython nbconvert "Batch Object Detection.ipynb"

['[NbConvertApp] Converting notebook Batch Object Detection.ipynb to html',
 '[NbConvertApp] Writing 211355 bytes to Batch Object Detection.html']

In [8]:
%sx mv "Batch Object Detection.html" $proc_files_dir

[]